# Image Processing Laboratory Notebooks <img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" alt="EPFL_logo" style="padding-right:10px;height:40px;float:left">

---

This Jupyter Notebook is part of a series of computer laboratories designed
to teach image-processing programming. They are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch),
    [Zhiyuan Hu](mailto:zhiyuan.hu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).

---

# Lab 2.1: Filtering
**Released**: Thursday, November 7, 2024

**Submission deadline**: Monday, November 18, 2024, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 2 (18 points), 10 % of the overall grade

<div class="alert alert-danger">
<b>Important:</b> don't forget to write down your name and SCIPER!
</div>

## Student Name: 
## SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
Run the next two cells to import the libraries and load the images that we will use throughout the lab.

In [ ]:
%use sos
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
import ipywidgets as widgets
from skimage import io
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = io.imread('images/bikesgray.tif').astype('float64')
camera = io.imread('images/camera-16bits.tif').astype('float64')
spots = io.imread('images/spots.tif').astype('float64')

In [ ]:
%use javascript
%get bikesgray camera spots
// import ImageAccess as Image
var Image = require('image-access')

# Filtering (9 points)

In this lab, you will review the basics of digital image filtering: how to implement a convolution (the main ingredient of filtering), separable and non-separable filters, filter design, and boundary conditions. The goal is to fully understand the filtering algorithm at a pixel-by-pixel level by implementing it in JavaScript, and learn the standard tools to use filtering in professional image-processing libraries in Python.

# Visualize images
Display and get familiar with the images.

In [ ]:
%use sos
image_list = [bikesgray, camera, spots]
imgs_viewer = viewer(image_list, widgets=True, hist=True)

# 1. Edge detection (3 points)

In this part of the lab, we will review one of the most basic image processing operation, edge detection. Moreover, we will use this filter to review the concepts of convolution and correlation. Edge detection algorithms try to detect sharp intesity differences between neighbouring pixels. A common vertical-edge detector (detecting changes in the $x$ direction) is the **Sobel filter**, represented by the impulse response

$$h[m,n] = 
\begin{bmatrix}
    -1 & 0 & 1 \\
    -2 & 0 & 2 \\
    -1 & 0 & 1
\end{bmatrix}\,.$$

When this filter is applied to a pixel, if the surrounding pixels have similar values, the output will be close to zero. However, if there is a significant difference between the values of the left and right columns, it suggests the presence of a vertical edge, resulting in a highly positive or highly negative intensity in the output.

As you saw in the lecture, there are filters that you can implement in a separable way, and filters that you cannot. Since the separable way is the most efficient one, you should develop the ability to recognize filters that are separable and implement them as such. Go through the next sections to implement both methods!

## 1.A. Non-separable implementation (1 point)

In the next cell we provide such a function `filterNonSeparable` that takes as input `img` and `mask`. This function however confuses correlation with convolution. **For 1 point**, find the mistake and correct it in the cell below.

<div class="alert alert-success">
    
<b>Hints</b>: <ul><li>The two-dimensional convolution between two images $h$ and $f$ is given by
$$(h \ast f)[x,y] = \sum_{m \in \mathbb{Z}}\sum_{n \in \mathbb{Z}}f[m,n]h[x-m,y-n]$$</li>
<li>In the formula above, $x-m$ is negative for some values but the indices of the JavaScript Array are all positive.</li>
</div>

In [ ]:
%use javascript

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparable(img, mask){
    var output = new Image(img.shape());
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(k, l);
                }
            }
            output.setPixel(x, y, val)
        }
    }
    return output
}

Run the next cell to perform a sanity check (convolution of the mask with an impulse image).

In [ ]:
// Lets do a sanity check
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);
var nonsymmetric_mask = new Image([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
// check the result
console.log('Convolving by an impulse should return the unchanged mask.\nMask:\n' + mask.visualize() )
console.log('\n' + nonsymmetric_mask.visualize())
console.log('Symmetric Result:\n' + filterNonSeparable(impulse, mask).visualize());
console.log('Nonsymmetric Result:\n' + filterNonSeparable(impulse, nonsymmetric_mask).visualize());
if(filterNonSeparable(impulse, mask).imageCompare(mask) == false){
    console.log('\nWARNING!!!\nThe function `filterNonSeparable` still provides an incorrect output.\n')
}else{
    console.log('It seems that the function works with a symmetric mask.')
    if (filterNonSeparable(impulse, nonsymmetric_mask).imageCompare(nonsymmetric_mask) == false){
        console.log('\nWARNING!!!\nIt seems that the function does not work with a nonsymmetric mask however.\n')
    }
    else{
        console.log('It seems that the function also works with a nonsymmetric mask')
    }
}

### Applying the function

Run the next cell to apply the `filterNonSeparable` function with the mask given above to the images `bikesgray`, `camera` and `spots`. 

In [ ]:
%use javascript
%get bikesgray camera spots

var vedge_bike_nons   = filterNonSeparable(new Image(bikesgray), mask).toArray();
var vedge_spots_nons  = filterNonSeparable(new Image(spots),     mask).toArray();
var vedge_camera_nons = filterNonSeparable(new Image(camera),    mask).toArray();

### Visualize the results
In the next cell we will visualize the results of edge detection. 

In [ ]:
%use sos
%get vedge_bike_nons vedge_camera_nons vedge_spots_nons --from javascript

image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

vedge_nons_viewer = viewer(image_list_nons, title=title_list, widgets=True)

## 1.B. Separable implementation (2 points)

In this subsection, you will modify the function `filterSeparable`. This function is based on the function `filter1D`, which takes as input a row/column of the image and a 1D mask and performs a 1D convolution. Recall that separable filters are divided into two 1D masks (`mx` and `my` in the code below). 

In the next cell, **for 1 point** complete the function `filterSeparable` by implementing the column filtering. The row filtering is provided and can be used as a reference.

In [ ]:
%use javascript

// function that performs a separable convolution on img using two 1D masks mx and my
function filterSeparable(img, mx, my){
    var output = new Image(img.shape());
    
    // iterate through every row 
    for(var y = 0; y < img.ny; y++){
        // extract row, apply filter and set column in output variable
        var row = img.getRow(y);
        var new_row = filter1D(row, mx)
        output.putRow(y, new_row);    
    }
    
    // iterate through every column
    for(var x = 0; x < img.nx; x++){
        // filter the columns
        
        // extract column, apply filter and set
        // YOUR CODE HERE
        
    }
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1); 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        output.setPixel(x, 0, val);
    }
    return output
}

### 1D masks

Now, before testing your separable implementation, **for 1 point**, code the 1D masks `mask_x` and `mask_y` in the cell below that corresponds to the separable version of the vertical-edge detection filter presented [at the beginning of this section](#1.-Edge-detection-(3-points)).

In [ ]:
%use javascript
%put mask_x mask_y
// change the masks to the correct value 
var mask_x = [0, 0, 0];
var mask_y = [0, 0, 0];

// YOUR CODE HERE

// convert the masks to Image objects
var mask_x_img = new Image([mask_x]);
var mask_y_img = new Image([mask_y]);

Now, run the next cell for a quick sanity check on your function `filterSeparable` and on your $1$D masks. We will convolve an impulse with your $1$D masks, and the result should be exactly the filter that we presented [at the beginning of the section](#1.-Edge-detection-(3-points)).

In [ ]:
%use javascript
// again lets do a sanity check with an impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare correct 2D mask
var mask = new Image([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]);

// First we check your masks with a low tolerance
if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-10) == false){
    // Repeat with high tolerance
    if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-4) == false){
    console.log('WARNING!!!\nSorry it looks like the separable filter or the 1D masks are not yet correct.');
    }else{
        console.log('WARNING!!!\nYour masks are almost correct, but the accuracy does not seem quite enough...');
}}else{
    console.log('The separable filter and the 1D masks have passed this sanity check!');
}

Now run the next cell to apply the function to the same three images we have been using before.

In [ ]:
%use javascript
%get bikesgray spots camera

var vedge_bike_s = filterSeparable(new Image(bikesgray), mask_x_img, mask_y_img).toArray();
var vedge_spots_s = filterSeparable(new Image(spots), mask_x_img, mask_y_img).toArray();
var vedge_camera_s = filterSeparable(new Image(camera), mask_x_img, mask_y_img).toArray();

Run the next one to visualize the results.

In [ ]:
%use sos
%get vedge_bike_s vedge_camera_s vedge_spots_s --from javascript

image_list_sep = [vedge_bike_s, vedge_camera_s, vedge_spots_s]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']
for i, image in enumerate(image_list_sep):
    image_list_sep[i] = np.array(image)

plt.close('all')
vedge_s_viewer = viewer(image_list_sep, title=title_list, widgets=True)

## 1.C. Comparison: Separable vs Non-separable

Are the results of the the separable and nonseparable implementations of a separable filter the same? If both your implementations are correct, the difference should be negligible. 

Explore the two histograms, zoom into different areas of the three images, and see if they are truly equal. There are several tests you can do to measure the equality of both implementations. For example, you can take the difference between the two images and visualize it (`image1 - image2`), or you can use the different functions provided by NumPy to compare arrays (see [numpy.testing.assert_array_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_equal.html) and [numpy.testing.assert_array_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html)). 

Use the next blank cell to do any necessary exploration. 


In [ ]:
%use sos

# YOUR CODE HERE

## 1.D. Edge detection in Python

In Python, the two functions `filterNonSeparable` and `filterSeparable` you implemented correspond to [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html) and [`convolve1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html) from the [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html) library. We will compare your implementation with them. Read the documentation of `convolve` and `convolve1d` and pay attention to the usage of these two functions in the next cell as you'll use these functions in the next section.

If you did everything correctly in Sections <a href="#1.A.-Non-separable-implementation-(1-point)">1.A.</a> and <a href = '#1.B.-Separable-implementation-(2-points)'>1.B.</a>, the next cell will display a winning message. Otherwise, it will plot your images with the regions that differ from the correct answer highlighted in red. You can uncomment the final line to visualize the four edge detections we have so far.

<div class="alert alert-info">

<font size="1"><b>Technical note on <code>dtype</code> (which stands for <i>data type</i>):</b><font> 
<font size="1"> In NumPy, when it comes to storing images, we primarily use three <code>dtype</code> : 
    <ul>
        <li><code>np.float64</code> ($64$-bit floating point)</li>
        <li><code>np.uint8</code> (8-bit <a href="https://en.wikipedia.org/wiki/Integer_(computer_science)" >unsigned integers</a> )</li>
        <li><code>np.int32</code> (32-bit signed integers)</li>
    </ul>
As you might have noticed from the <a href="#Imports">imports</a>, in this lab, we store our images as <code>np.float64</code> due to its versatility. However, it's important to note that most images are typically stored as <code>np.uint8</code>. This can become problematic when dealing with operations where values may exceed the 0 to 255 range (such as gradient calculations). To prevent potential issues and bugs, it is crucial to specify the appropriate <code>dtype</code> for outputs in such scenarios. By default, the <code>dtype</code> of the output array inherits from the input array.
   <br> 
Additionally, if our computations involve only integers, we can optimize memory usage and improve performance by using <code>np.int32</code> instead of <code>np.float64</code>.
</div>

In [ ]:
%use sos 

# Apply non-separable Filter
mask = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
vedge_ndi = ndi.convolve(bikesgray, mask, output=np.int32)

# Apply separable Filter 
vedge_ndi_s = ndi.convolve1d(bikesgray, mask_y, axis=0, output=np.float64) # filter the columns
vedge_ndi_s = ndi.convolve1d(vedge_ndi_s, mask_x, axis=1, output=np.float64) # filter the rows

# Visualize if necessary
plt.close('all')
if np.allclose(vedge_ndi, vedge_bike_nons):
    print('Congratulations! Your non-separable implementation is correct!')
else :
    print('Review your non-separable JS implementation!')
    viewer([np.array(vedge_bike_nons), vedge_ndi], title=['JS non-separable', 'Python non-separable'], compare=True)

if np.allclose(vedge_ndi_s, vedge_bike_s):
    print('Congratulations! Your separable filter is correct!')
else :
    print('Review your separable JS implementation. \
           \nThe problem could be either on your 1D masks, or on the implementation itself. The previous sanity checks and the red areas should guide you.')
    viewer([np.array(vedge_bike_s), vedge_ndi_s], title=['JS separable', 'Python separable'], compare=True)

# vedge_viewer = viewer([np.array(vedge_bike_nons), vedge_ndi, np.array(vedge_bike_s), vedge_ndi_s], title=['JS non-separable', 'Python non-separable', 'JS separable', 'Python separable'], subplots=(2,2))

# 2. Implementation and classification of digital filters (6 points)

In this exercise, we will practice the implementation and classification of digital filters. Moreover, you will practice separable and non-separable implementations in Python. You'll implement 6 functions in Python that correspond to 6 different digital filters. Each of them takes an input image and outputs the filtered version of it. You need to
* hard code the mask inside the function,
* apply the filtering using either `ndi.convolve` and `ndi.convolve1d`, depending on if the filter is separable.

You can define 1D and 2D filters as 1D and 2D numpy arrays, e.g.
```python
mask = np.array([1, 2])
```
```python
mask = np.array([[ 1,  2], 
                 [ 6,  7]])
```

Implementing each function is worth **0.5 points**, classifing their visual effect is worth **0.5 points**.

<div class="alert alert-danger">
<b>Important:</b> If a mask is separable and you don't implement it as such, you'll not get the points.
</div>

<div class="alert alert-success">
    <font size="2"><b>Note on the separability of filters:</b> <br>Some filters, in particular simple edge detector filters can intuitively be recognized as separable. Furthermore, their $1\mathrm{D}$ masks can also be intuitively proposed. From these $1\mathrm{D}$ masks, a simple test consists in taking the matrix product between them, which should result in the original filter. However, there is a more systematic approach.<br>
    As you should know, any matrix $\mathbf{\mathrm{M}}\in\mathbb{R}^{m\times n}$ can be factorized into its components $\mathbf{\mathrm{USV^{T}}}$ by Singular Value Decomposition, where $\mathbf{\mathrm{U}}\in\mathbb{R}^{m\times m}$, $\mathbf{\mathrm{S}}\in\mathbb{R}^{\min(m, n)\times \min(m, n)}$ corresponds to a diagonal matrix with the singular values of $\mathbf{\mathrm{M}}$, and $\mathbf{\mathrm{V}}\in\mathbb{R}^{n\times n}$. It follows from the definition of the matrix product that if only one singular value is larger than zero, the filter is separable.<br>
    Nonetheless, for simple filters, we still recommend you to use the <i>intuitive</i> filters. If you want to extract the filters using SVD, you can use the <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html"><code>svd</code></a> function of the NumPy Linear Algebra package. We leave you the task of extracting the actual $1\mathrm{D}$ masks from it.<font> 
</div>

## 2.A. Mask A (1 point)

```python3
    [ 0,  0,   1,  0,  0]
    [ 0,  1,   2,  1,  0]
A = [ 1,  2, -16,  2,  1]
    [ 0,  1,   2,  1,  0]
    [ 0,  0,   1,  0,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask A
def filter_A(img):
    output = img.copy()
       
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_A` to the image `bikesgray`.

<div class = "alert alert-info"><b>Note</b>: If you cannot see the whole <code>viewer</code>, remember that you can use <code>Ctrl</code>+<code>b</code> to hide the sidebar at the left. If you have already done this, you can always use <code>Ctrl</code> + <code>-</code> to zoom out a bit</div>

In [ ]:
%use sos

bikesgray_A = filter_A(bikesgray)
plt.close('all')
vis_A = viewer([bikesgray, bikesgray_A], title=['Original', 'Convolved with filter A'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.B. Mask B (1 point)

```python3
    [ 0,  1,  2,  1,  0]
    [ 1,  2,  4,  2,  1]
B = [ 2,  4,  8,  4,  2]
    [ 1,  2,  4,  2,  1]
    [ 0,  1,  2,  1,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask B
def filter_B(img):
    output = img.copy()
       
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_B` to the image `bikesgray`.

In [ ]:
%use sos

bikesgray_B = filter_B(bikesgray)
plt.close('all')
vis_B = viewer([bikesgray, bikesgray_B], title=['Original', 'Filtered with mask B'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.C. Mask C (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_C`, which convolves the input image with the filter 

```python3
    [-1,  -1,  -1,  -1,  -1]
    [-2,  -2,  -2,  -2,  -2]
C = [ 0,   0,   0,   0,   0]
    [ 2,   2,   2,   2,   2]
    [ 1,   1,   1,   1,   1]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask C
def filter_C(img):
    output = img.copy()
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_C` to the image `bikesgray`.

In [ ]:
%use sos

bikesgray_C = filter_C(bikesgray)
plt.close('all')
vis_C = viewer([bikesgray, bikesgray_C], title=['Original', 'Filtered with mask C'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.D. Mask D (1 point)

```python3
    [-1,  -4,  -6,  -4, -1]
    [-4, -16, -24, -16, -4]
D = [-6, -24, 476, -24, -6]
    [-4, -16, -24, -16, -4]
    [-1,  -4,  -6,  -4, -1]
```

In [ ]:
%use sos

# Function that filters img with the mask D
def filter_D(img):
    output = img.copy()
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_D` to the image `bikesgray`.

In [ ]:
%use sos

bikesgray_D = filter_D(bikesgray)
plt.close('all')
vis_D = viewer([bikesgray, bikesgray_D], title=['Original', 'Filtered with mask D'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.E. Mask E (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_E`, which convolves the input image with the filter 

```python3
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
E = [0, 0, 1, 0, 0]
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
```
Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask E
def filter_E(img):
    output = img.copy()
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_E` to the image `bikesgray`.

In [ ]:
%use sos

bikesgray_E = filter_E(bikesgray)
plt.close('all')
vis_E = viewer([bikesgray, bikesgray_E], title=['Original', 'Filtered with mask E'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.F. Mask F (1 point)

```python3
    [ 0, -1, -1,  0,  0]
    [-1, -1,  0,  0,  0]
F = [-1,  0,  0,  0,  1]
    [ 0,  0,  0,  1,  1]
    [ 0,  0,  1,  1,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask F
def filter_F(img):
    output = img.copy()
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_F` to the image `bikesgray`.

In [ ]:
%use sos

bikesgray_F = filter_F(bikesgray)
plt.close('all')
vis_F = viewer([bikesgray, bikesgray_F], title=['Original', 'Filtered with mask F'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring
2. vertical blurring
3. horizontal blurring
4. diagonal blurring
5. sharpening
6. vertical-edge detection
7. horizontal-edge detection
8. diagonal edge detection
9. Laplacian: a second order derivative to find edges (maxima and minima)
10. it has not effect

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

<div class="alert alert-success">
    
<p><b>Congratulations on finishing the first part of the digital filtering lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1111434">Moodle</a>, in a zip file together with the second part of this lab.
</p>
</div>

* Keep the name of the notebook as: *1_Filtering.ipynb*,
* Name the zip file: *Filtering_lab.zip*.